<a href="https://colab.research.google.com/github/pranjalchaubey/Video-Cutter/blob/main/Jupyter%20Notebook/Video_Cutter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install moviepy
!pip install ffmpeg-python
!apt-get update && apt-get install -y libx265-dev

Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:4 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:6 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:9 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,517 kB]
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [2,830 kB]
Get:13 http://archive.ubuntu.com/ubuntu jammy-updates/restricted amd

In [ ]:
from moviepy.editor import VideoFileClip
from google.colab import files
import matplotlib.pyplot as plt
import numpy as np
import os
import cv2
import time
import ffmpeg

  if event.key is 'enter':



In [ ]:
# Mount Google Drive
try:
    from google.colab import drive
    from google.colab import files
    drive.mount('/content/drive')
except:
    print("Not running in Colab environment")

Mounted at /content/drive


In [ ]:
# Variables to configure - UPDATE THESE WITH YOUR FILE PATHS
video_path = '/content/DMS 2024.MP4'  # Full path to video in Google Drive
output_filename = "new_clip.mp4"  # Name of the new clip



In [ ]:
def timestamp_to_seconds(timestamp):
    """Convert timestamp string to seconds"""
    h, m, s = timestamp.split(':')
    hours = int(h)
    minutes = int(m)
    seconds = float(s)
    return hours * 3600 + minutes * 60 + seconds

In [ ]:
def display_frame(timestamp, figsize=(15, 10), target_width=320):
    """
    Display a single frame from the video at the specified timestamp directly in the notebook

    Args:
        timestamp (str): Timestamp in format "HH:MM:SS.fff"
        figsize (tuple): Size of the displayed figure (width, height)
        target_width (int): Width to resize the frame to (maintaining aspect ratio)
    """
    try:
        # Convert timestamp to seconds
        frame_time = timestamp_to_seconds(timestamp)

        # Load the video
        video = VideoFileClip(video_path)

        # Extract the frame
        frame = video.get_frame(frame_time)

        # Convert frame to uint8 if it's not already
        if frame.dtype != np.uint8:
            frame = (frame * 255).astype(np.uint8)

        # Calculate new dimensions maintaining aspect ratio
        height, width = frame.shape[:2]
        aspect_ratio = width / height
        target_height = int(target_width / aspect_ratio)

        # Resize frame
        frame_resized = cv2.resize(frame, (target_width, target_height), interpolation=cv2.INTER_AREA)

        # Display the frame
        plt.figure(figsize=figsize)
        plt.imshow(cv2.cvtColor(frame_resized, cv2.COLOR_BGR2RGB))
        plt.axis('off')  # Hide axes
        plt.title(f'Frame at {timestamp} (Resolution: {target_width}x{target_height})')
        plt.show()

        # Clean up
        video.close()

        print(f"Successfully displayed frame at {timestamp}")

    except Exception as e:
        print(f"An error occurred while displaying frame: {str(e)}")

def display_frame2(timestamp, figsize=(15, 10)):
    """
    Display a single frame from the video at the specified timestamp directly in the notebook

    Args:
        timestamp (str): Timestamp in format "HH:MM:SS.fff"
        figsize (tuple): Size of the displayed figure (width, height)
    """
    try:
        # Convert timestamp to seconds
        frame_time = timestamp_to_seconds(timestamp)

        # Load the video
        video = VideoFileClip(video_path)

        # Extract the frame
        frame = video.get_frame(frame_time)

        # Display the frame
        plt.figure(figsize=figsize)
        plt.imshow(frame)
        plt.axis('off')  # Hide axes
        plt.title(f'Frame at {timestamp}')
        plt.show()

        # Clean up
        video.close()

        print(f"Successfully displayed frame at {timestamp}")

    except Exception as e:
        print(f"An error occurred while displaying frame: {str(e)}")

In [ ]:
def display_multiple_frames(timestamps, rows=None, figsize=(15, 10), target_width=320):
    """
    Display multiple frames in a grid

    Args:
        timestamps (list): List of timestamps in "HH:MM:SS.fff" format
        rows (int): Number of rows in the grid (optional)
        figsize (tuple): Size of the entire figure
        target_width (int): Width to resize each frame to (maintaining aspect ratio)
    """
    n = len(timestamps)
    if rows is None:
        rows = int(np.ceil(np.sqrt(n)))
    cols = int(np.ceil(n / rows))

    video = VideoFileClip(video_path)

    fig, axes = plt.subplots(rows, cols, figsize=figsize)
    if rows == 1 and cols == 1:
        axes = np.array([axes])
    axes = axes.flatten()

    for i, timestamp in enumerate(timestamps):
        if i < n:
            frame_time = timestamp_to_seconds(timestamp)
            frame = video.get_frame(frame_time)

            # Convert frame to uint8 if it's not already
            if frame.dtype != np.uint8:
                frame = (frame * 255).astype(np.uint8)

            # Calculate new dimensions maintaining aspect ratio
            height, width = frame.shape[:2]
            aspect_ratio = width / height
            target_height = int(target_width / aspect_ratio)

            # Resize frame
            frame_resized = cv2.resize(frame, (target_width, target_height), interpolation=cv2.INTER_AREA)

            axes[i].imshow(cv2.cvtColor(frame_resized, cv2.COLOR_BGR2RGB))
            axes[i].axis('off')
            axes[i].set_title(f'Frame at {timestamp}\n({target_width}x{target_height})')

    # Hide empty subplots
    for i in range(n, len(axes)):
        axes[i].axis('off')

    plt.tight_layout()
    plt.show()
    video.close()


# def display_multiple_frames(timestamps, rows=None, figsize=(15, 10)):
#     """
#     Display multiple frames in a grid

#     Args:
#         timestamps (list): List of timestamps in "HH:MM:SS.fff" format
#         rows (int): Number of rows in the grid (optional)
#         figsize (tuple): Size of the entire figure
#     """
#     n = len(timestamps)
#     if rows is None:
#         rows = int(np.ceil(np.sqrt(n)))
#     cols = int(np.ceil(n / rows))

#     video = VideoFileClip(video_path)

#     fig, axes = plt.subplots(rows, cols, figsize=figsize)
#     if rows == 1 and cols == 1:
#         axes = np.array([axes])
#     axes = axes.flatten()

#     for i, timestamp in enumerate(timestamps):
#         if i < n:
#             frame_time = timestamp_to_seconds(timestamp)
#             frame = video.get_frame(frame_time)
#             axes[i].imshow(frame)
#             axes[i].axis('off')
#             axes[i].set_title(f'Frame at {timestamp}')

#     # Hide empty subplots
#     for i in range(n, len(axes)):
#         axes[i].axis('off')

#     plt.tight_layout()
#     plt.show()
#     video.close()


In [ ]:
def display_frames_between(start_time, end_time=None, interval_ms=5, rows=None, target_width=320):
    """
    Display frames between start and end time at specified millisecond intervals.
    If end_time is not specified, shows next 50 frames from start_time.

    Args:
        start_time (str): Start timestamp in format "HH:MM:SS.fff"
        end_time (str, optional): End timestamp in format "HH:MM:SS.fff"
        interval_ms (int): Interval between frames in milliseconds (default: 5)
        rows (int): Number of rows in the grid (optional)
        target_width (int): Width to resize each frame to (maintaining aspect ratio)
    """
    # If end_time is not specified, calculate it for 50 frames
    if end_time is None:
        start_seconds = timestamp_to_seconds(start_time)
        # Calculate end time (50 frames * interval_ms milliseconds)
        end_seconds = start_seconds + (50 * interval_ms / 1000)

        # Convert end_seconds to timestamp format
        hours = int(end_seconds // 3600)
        minutes = int((end_seconds % 3600) // 60)
        seconds = end_seconds % 60
        end_time = f"{hours:02d}:{minutes:02d}:{seconds:06.3f}"
        print(f"Auto-generated end time: {end_time}")

    # Convert times to milliseconds
    start_seconds = timestamp_to_seconds(start_time)
    end_seconds = timestamp_to_seconds(end_time)

    start_ms = int(start_seconds * 1000)
    end_ms = int(end_seconds * 1000)

    # Generate timestamps
    timestamps = []
    current_ms = start_ms
    while current_ms <= end_ms:
        total_seconds = current_ms / 1000
        hours = int(total_seconds // 3600)
        minutes = int((total_seconds % 3600) // 60)
        seconds = total_seconds % 60
        timestamp = f"{hours:02d}:{minutes:02d}:{seconds:06.3f}"
        timestamps.append(timestamp)
        current_ms += interval_ms

    num_frames = len(timestamps)
    print(f"Generating {num_frames} frames from {start_time} to {end_time}")

    # Optimize grid layout and figure size
    if rows is None:
        # Calculate optimal number of rows for a roughly square grid
        rows = int(np.ceil(np.sqrt(num_frames)))
    cols = int(np.ceil(num_frames / rows))

    # Calculate optimal figure size based on number of columns and rows
    # We'll use a base size of 2.5 inches per image
    figsize = (2.5 * cols, 2.5 * rows)

    video = VideoFileClip(video_path)

    fig, axes = plt.subplots(rows, cols, figsize=figsize)
    if rows == 1 and cols == 1:
        axes = np.array([axes])
    axes = axes.flatten()

    for i, timestamp in enumerate(timestamps):
        if i < num_frames:
            frame_time = timestamp_to_seconds(timestamp)
            frame = video.get_frame(frame_time)

            # Convert frame to uint8 if needed
            if frame.dtype != np.uint8:
                frame = (frame * 255).astype(np.uint8)

            # Calculate new dimensions maintaining aspect ratio
            height, width = frame.shape[:2]
            aspect_ratio = width / height
            target_height = int(target_width / aspect_ratio)

            # Resize frame
            frame_resized = cv2.resize(frame, (target_width, target_height),
                                     interpolation=cv2.INTER_AREA)

            axes[i].imshow(cv2.cvtColor(frame_resized, cv2.COLOR_BGR2RGB))
            axes[i].axis('off')

            # Format timestamp for display (show only relevant part)
            display_timestamp = timestamp.split(':', 2)[2]  # Show only seconds part
            axes[i].set_title(f'{display_timestamp}s', fontsize=8)

    # Hide empty subplots
    for i in range(num_frames, len(axes)):
        axes[i].axis('off')

    plt.tight_layout()
    plt.show()
    video.close()

# Example usage:
# For 50 frames from start time:
# display_frames_between("00:00:20.620")

# For specific range:
# display_frames_between("00:00:20.620", "00:00:20.645")


# def display_frames_between(start_time, end_time, interval_ms=5, rows=None, figsize=(15, 10), target_width=320):
#     """
#     Display frames between start and end time at specified millisecond intervals

#     Args:
#         start_time (str): Start timestamp in format "HH:MM:SS.fff"
#         end_time (str): End timestamp in format "HH:MM:SS.fff"
#         interval_ms (int): Interval between frames in milliseconds (default: 5)
#         rows (int): Number of rows in the grid (optional)
#         figsize (tuple): Size of the entire figure
#         target_width (int): Width to resize each frame to (maintaining aspect ratio)
#     """
#     # Convert start and end times to milliseconds
#     start_seconds = timestamp_to_seconds(start_time)
#     end_seconds = timestamp_to_seconds(end_time)

#     start_ms = int(start_seconds * 1000)
#     end_ms = int(end_seconds * 1000)

#     # Generate timestamps at specified intervals
#     timestamps = []
#     current_ms = start_ms
#     while current_ms <= end_ms:
#         # Convert milliseconds back to timestamp format
#         total_seconds = current_ms / 1000
#         hours = int(total_seconds // 3600)
#         minutes = int((total_seconds % 3600) // 60)
#         seconds = total_seconds % 60
#         timestamp = f"{hours:02d}:{minutes:02d}:{seconds:06.3f}"
#         timestamps.append(timestamp)
#         current_ms += interval_ms

#     print(f"Generated {len(timestamps)} frames from {start_time} to {end_time}")

#     # Use existing display_multiple_frames function
#     display_multiple_frames(timestamps, rows, figsize, target_width)

# # Example usage:
# # display_frames_between("00:00:20.620", "00:00:20.645")

In [ ]:
# def cut_video(start_time, end_time, output_filename):
#     """Cut video between specified timestamps and save to Google Drive, preserving HDR"""
#     try:
#         # Convert timestamps to seconds for moviepy
#         start_seconds = timestamp_to_seconds(start_time)
#         end_seconds = timestamp_to_seconds(end_time)

#         # Create output path in Google Drive
#         output_path = f"/content/drive/MyDrive/DMS2024/{output_filename}"

#         # Check if output directory exists
#         output_dir = os.path.dirname(output_path)
#         if not os.path.exists(output_dir):
#             print(f"Error: Output directory {output_dir} does not exist")
#             return

#         print(f"Cutting video from {start_time} to {end_time}...")
#         print(f"Saving to {output_path} with HDR preservation...")

#         # Input video stream
#         stream = ffmpeg.input(video_path, ss=start_seconds, t=end_seconds-start_seconds)

#         # Output stream with HDR metadata and color space preservation
#         stream = ffmpeg.output(stream, output_path,
#                              vcodec='libx264',                    # Use H.264 codec
#                              acodec='aac',                        # AAC for audio
#                              video_bitrate='20M',                 # High bitrate for quality
#                              color_primaries='bt2020',            # BT.2020 color primaries for HDR
#                              color_trc='smpte2084',              # PQ transfer characteristics
#                              colorspace='bt2020nc',               # BT.2020 color space
#                              pix_fmt='yuv420p10le',              # 10-bit color depth
#                              x264opts='hdr-opt=1:repeat-headers=1:colorprim=bt2020:transfer=smpte2084:colormatrix=bt2020nc',
#                              preset='slow',                       # Slower encoding for better quality
#                              movflags='+faststart')              # Fast start for streaming

#         # Run the ffmpeg command
#         ffmpeg.run(stream, overwrite_output=True)

#         print(f"Successfully created HDR clip at: {output_path}")

#     except Exception as e:
#         print(f"An error occurred: {str(e)}")


# def cut_video(start_time, end_time, output_filename):
#     """Cut video between specified timestamps and save to Google Drive"""
#     try:
#         # Convert timestamps to seconds for moviepy
#         start_seconds = timestamp_to_seconds(start_time)
#         end_seconds = timestamp_to_seconds(end_time)

#         # Load the video file
#         video = VideoFileClip(video_path)

#         # Cut the video
#         cut_video = video.subclip(start_seconds, end_seconds)

#         # Create output path in Google Drive
#         output_path = f"/content/drive/MyDrive/DMS2024/{output_filename}"

#         # Write the video
#         cut_video.write_videofile(
#             output_path,
#             codec='libx264',
#             audio_codec='aac',
#             temp_audiofile='temp-audio.m4a',
#             remove_temp=True
#         )

#         # Clean up
#         video.close()
#         cut_video.close()

#         print(f"Successfully created clip at: {output_path}")

#     except Exception as e:
#         print(f"An error occurred: {str(e)}")

In [ ]:
# def cut_video(start_time, end_time, output_filename):
#     """Cut video between specified timestamps and save to Google Drive, preserving HDR"""
#     try:
#         # Convert timestamps to seconds for moviepy
#         start_seconds = timestamp_to_seconds(start_time)
#         end_seconds = timestamp_to_seconds(end_time)

#         # Create output path in Google Drive
#         output_path = f"/content/drive/MyDrive/DMS2024/{output_filename}"

#         # Check if output directory exists
#         output_dir = os.path.dirname(output_path)
#         if not os.path.exists(output_dir):
#             print(f"Error: Output directory {output_dir} does not exist")
#             return

#         print(f"Cutting video from {start_time} to {end_time}...")
#         print(f"Saving to {output_path} with HDR preservation...")

#         try:
#             # Input video stream
#             stream = ffmpeg.input(video_path, ss=start_seconds, t=end_seconds-start_seconds)

#             # Output stream with HDR metadata and color space preservation
#             stream = ffmpeg.output(stream, output_path,
#                                  vcodec='libx264',                    # Use H.264 codec
#                                  acodec='aac',                        # AAC for audio
#                                  video_bitrate='20M',                 # High bitrate for quality
#                                  color_primaries='bt2020',            # BT.2020 color primaries for HDR
#                                  color_trc='smpte2084',              # PQ transfer characteristics
#                                  colorspace='bt2020nc',               # BT.2020 color space
#                                  pix_fmt='yuv420p10le',              # 10-bit color depth
#                                  x264opts='hdr-opt=1:repeat-headers=1:colorprim=bt2020:transfer=smpte2084:colormatrix=bt2020nc',
#                                  preset='slow',                       # Slower encoding for better quality
#                                  movflags='+faststart')              # Fast start for streaming

#             # Print the ffmpeg command that will be executed
#             print("\nFFmpeg command:")
#             print(' '.join(ffmpeg.compile(stream)))

#             # Run the ffmpeg command and capture output
#             stdout, stderr = ffmpeg.run(stream, capture_stdout=True, capture_stderr=True, overwrite_output=True)

#         except ffmpeg.Error as e:
#             print("\nFFmpeg error output:")
#             print(e.stderr.decode())
#             raise

#         print(f"Successfully created HDR clip at: {output_path}")

#     except Exception as e:
#         print(f"An error occurred: {str(e)}")

In [ ]:
def analyze_video(video_path):
    """
    Analyze video file and return its HDR and encoding properties
    """
    try:
        # Get video information using ffmpeg probe
        probe = ffmpeg.probe(video_path)
        video_info = next(s for s in probe['streams'] if s['codec_type'] == 'video')

        # Extract relevant information
        info = {
            'codec': video_info.get('codec_name', ''),
            'pixel_format': video_info.get('pix_fmt', ''),
            'color_space': video_info.get('color_space', ''),
            'color_transfer': video_info.get('color_transfer', ''),
            'color_primaries': video_info.get('color_primaries', ''),
            'bit_rate': video_info.get('bit_rate', ''),
            'profile': video_info.get('profile', ''),
            'width': video_info.get('width', ''),
            'height': video_info.get('height', ''),
            'fps': eval(video_info.get('r_frame_rate', '0/1')),  # Convert fraction to number
        }

        print("\nVideo Analysis Results:")
        print(f"Codec: {info['codec']}")
        print(f"Pixel Format: {info['pixel_format']}")
        print(f"Color Space: {info['color_space']}")
        print(f"Color Transfer: {info['color_transfer']}")
        print(f"Color Primaries: {info['color_primaries']}")
        print(f"Bit Rate: {int(info['bit_rate'])/1000000:.2f} Mbps" if info['bit_rate'] else "Bit Rate: Unknown")
        print(f"Profile: {info['profile']}")
        print(f"Resolution: {info['width']}x{info['height']}")
        print(f"Frame Rate: {info['fps']:.2f} fps")

        # Check for HDR metadata
        if 'side_data_list' in video_info:
            print("\nHDR Metadata:")
            for side_data in video_info['side_data_list']:
                if side_data['side_data_type'] == 'Content light level metadata':
                    print(f"MaxCLL: {side_data.get('max_content', 'Unknown')}")
                    print(f"MaxFALL: {side_data.get('max_average', 'Unknown')}")
                if side_data['side_data_type'] == 'Mastering display metadata':
                    print(f"Display Primaries: {side_data.get('red_x', 'Unknown')},{side_data.get('red_y', 'Unknown')}")
                    print(f"White Point: {side_data.get('white_point_x', 'Unknown')},{side_data.get('white_point_y', 'Unknown')}")
                    print(f"Min/Max Luminance: {side_data.get('min_luminance', 'Unknown')}/{side_data.get('max_luminance', 'Unknown')}")

        # Print any additional useful metadata
        if 'tags' in video_info:
            print("\nAdditional Metadata:")
            for key, value in video_info['tags'].items():
                print(f"{key}: {value}")

        return info

    except Exception as e:
        print(f"Error analyzing video: {str(e)}")
        return None

In [ ]:
# # Global configuration to store video analysis results
# class VideoConfig:
#     def __init__(self):
#         self.is_analyzed = False
#         self.encoding_params = None

#     def analyze_and_store(self, video_path):
#         """Analyze video and store parameters for future use"""
#         if self.is_analyzed:
#             return self.encoding_params

#         info = analyze_video(video_path)
#         if info:
#             self.encoding_params = {
#                 'vcodec': 'libx265',  # Using libx265 instead of libhevc for better compatibility
#                 'pix_fmt': info['pixel_format'],
#                 'video_bitrate': str(int(int(info['bit_rate']) * 1.1)),  # 10% higher for safety
#                 'colorspace': info['color_space'],
#                 'color_trc': info['color_transfer'],
#                 'color_primaries': info['color_primaries'],
#                 'preset': 'slow',
#                 'x265_params': f"hdr-opt=1:repeat-headers=1:colorprim={info['color_primaries']}:"
#                               f"transfer={info['color_transfer']}:colormatrix={info['color_space']}"
#             }
#             self.is_analyzed = True
#             return self.encoding_params
#         return None

# # Create global instance
# video_config = VideoConfig()


In [ ]:
# def initialize_video_config(video_path):
#     """Initialize video configuration once per session"""
#     return video_config.analyze_and_store(video_path)

# def cut_video(start_time, end_time, output_filename):
#     """Cut video between specified timestamps and save to Google Drive, preserving HDR"""
#     try:
#         # Ensure we have video configuration
#         if not video_config.is_analyzed:
#             if not initialize_video_config(video_path):
#                 print("Could not initialize video configuration")
#                 return

#         # Convert timestamps to seconds
#         start_seconds = timestamp_to_seconds(start_time)
#         end_seconds = timestamp_to_seconds(end_time)

#         # Create output path in Google Drive
#         output_path = f"/content/drive/MyDrive/DMS2024/{output_filename}"

#         # Check if output directory exists
#         output_dir = os.path.dirname(output_path)
#         if not os.path.exists(output_dir):
#             print(f"Error: Output directory {output_dir} does not exist")
#             return

#         print(f"Cutting video from {start_time} to {end_time}...")
#         print(f"Saving to {output_path} with HDR preservation...")

#         try:
#             # Input video stream
#             stream = ffmpeg.input(video_path, ss=start_seconds, t=end_seconds-start_seconds)

#             # Output stream with HDR metadata and color space preservation
#             stream = ffmpeg.output(stream, output_path,
#                                  **video_config.encoding_params,  # Unpack stored encoding parameters
#                                  acodec='aac',                   # Audio codec
#                                  movflags='+faststart')          # Fast start for streaming

#             # Print the ffmpeg command
#             print("\nFFmpeg command:")
#             print(' '.join(ffmpeg.compile(stream)))

#             # Run the ffmpeg command
#             stdout, stderr = ffmpeg.run(stream, capture_stdout=True, capture_stderr=True, overwrite_output=True)

#         except ffmpeg.Error as e:
#             print("\nFFmpeg error output:")
#             print(e.stderr.decode())
#             raise

#         print(f"Successfully created HDR clip at: {output_path}")

#     except Exception as e:
#         print(f"An error occurred: {str(e)}")

In [ ]:

# class VideoConfig:
#     def __init__(self):
#         self.is_analyzed = False
#         self.encoding_params = None

#     def analyze_and_store(self, video_path):
#         """Analyze video and store parameters for future use"""
#         if self.is_analyzed:
#             return self.encoding_params

#         info = analyze_video(video_path)
#         if info:
#             # Convert bitrate to string with 'k' suffix for kbps
#             bitrate_kbps = str(int(int(info['bit_rate']) / 1000)) + 'k'

#             self.encoding_params = {
#                 'vcodec': 'libx265',
#                 'pix_fmt': info['pixel_format'],
#                 'video_bitrate': bitrate_kbps,
#                 'colorspace': info['color_space'],
#                 'color_trc': info['color_transfer'],
#                 'color_primaries': info['color_primaries'],
#                 'preset': 'slow',
#                 # Instead of x265_params, use individual parameters
#                 'x265-params': f'hdr-opt=1:repeat-headers=1:colorprim={info["color_primaries"]}:'
#                               f'transfer={info["color_transfer"]}:colormatrix={info["color_space"]}'
#             }
#             self.is_analyzed = True
#             return self.encoding_params
#         return None

# # Global instance
# video_config = VideoConfig()

# def initialize_video_config(video_path):
#     """Initialize video configuration once per session"""
#     return video_config.analyze_and_store(video_path)

# def cut_video(start_time, end_time, output_filename):
#     """Cut video between specified timestamps and save to Google Drive, preserving HDR"""
#     try:
#         # Ensure we have video configuration
#         if not video_config.is_analyzed:
#             if not initialize_video_config(video_path):
#                 print("Could not initialize video configuration")
#                 return

#         # Convert timestamps to seconds
#         start_seconds = timestamp_to_seconds(start_time)
#         end_seconds = timestamp_to_seconds(end_time)

#         # Create output path in Google Drive
#         output_path = f"/content/drive/MyDrive/DMS2024/{output_filename}"

#         # Check if output directory exists
#         output_dir = os.path.dirname(output_path)
#         if not os.path.exists(output_dir):
#             print(f"Error: Output directory {output_dir} does not exist")
#             return

#         print(f"Cutting video from {start_time} to {end_time}...")
#         print(f"Saving to {output_path} with HDR preservation...")

#         try:
#             # Input video stream
#             stream = ffmpeg.input(video_path, ss=start_seconds, t=end_seconds-start_seconds)

#             # Create a copy of encoding params to modify
#             encode_params = video_config.encoding_params.copy()

#             # Extract x265-params and remove from dict
#             x265_params = encode_params.pop('x265-params', '')

#             # Output stream with HDR metadata and color space preservation
#             stream = ffmpeg.output(stream,
#                                  output_path,
#                                  acodec='aac',
#                                  movflags='+faststart',
#                                  **encode_params,
#                                  **{'-x265-params': x265_params}  # Add x265-params separately
#                                  )

#             # Print the ffmpeg command
#             print("\nFFmpeg command:")
#             print(' '.join(ffmpeg.compile(stream)))

#             # Run the ffmpeg command
#             stdout, stderr = ffmpeg.run(stream, capture_stdout=True, capture_stderr=True, overwrite_output=True)

#         except ffmpeg.Error as e:
#             print("\nFFmpeg error output:")
#             print(e.stderr.decode())
#             raise

#         print(f"Successfully created HDR clip at: {output_path}")

#     except Exception as e:
#         print(f"An error occurred: {str(e)}")

In [ ]:
class VideoConfig:
    def __init__(self):
        self.is_analyzed = False
        self.encoding_params = None

    def analyze_and_store(self, video_path):
        """Analyze video and store parameters for future use"""
        if self.is_analyzed:
            return self.encoding_params

        info = analyze_video(video_path)
        if info:
            self.encoding_params = {
                'c:v': 'libx265',                    # Video codec
                'c:a': 'aac',                        # Audio codec
                'pix_fmt': info['pixel_format'],     # Pixel format
                'b:v': f"{int(int(info['bit_rate'])/1000)}k",  # Video bitrate in kbps
                'preset': 'slow',                    # Encoding preset
                'profile:v': 'main10',              # 10-bit profile
                'movflags': '+faststart',           # Stream optimization
                # Color space parameters
                'colorspace': info['color_space'],
                'color_primaries': info['color_primaries'],
                'color_trc': info['color_transfer']
            }
            self.is_analyzed = True
            return self.encoding_params
        return None

# Global instance
video_config = VideoConfig()

def initialize_video_config(video_path):
    """Initialize video configuration once per session"""
    return video_config.analyze_and_store(video_path)

def cut_video(start_time, end_time, output_filename):
    """Cut video between specified timestamps and save to Google Drive, preserving HDR"""
    try:
        # Ensure we have video configuration
        if not video_config.is_analyzed:
            if not initialize_video_config(video_path):
                print("Could not initialize video configuration")
                return

        # Convert timestamps to seconds
        start_seconds = timestamp_to_seconds(start_time)
        end_seconds = timestamp_to_seconds(end_time)

        # Create output path in Google Drive
        output_path = f"/content/drive/MyDrive/DMS2024HD/{output_filename}"

        # Check if output directory exists
        output_dir = os.path.dirname(output_path)
        if not os.path.exists(output_dir):
            print(f"Error: Output directory {output_dir} does not exist")
            return

        print(f"Cutting video from {start_time} to {end_time}...")
        print(f"Saving to {output_path} with HDR preservation...")

        try:
            # Input video stream
            stream = ffmpeg.input(video_path, ss=start_seconds, t=end_seconds-start_seconds)

            # Output stream with HDR metadata and color space preservation
            stream = ffmpeg.output(
                stream,
                output_path,
                **video_config.encoding_params
            )

            # Print the ffmpeg command
            print("\nFFmpeg command:")
            print(' '.join(ffmpeg.compile(stream)))

            # Run the ffmpeg command
            stdout, stderr = ffmpeg.run(stream, capture_stdout=True, capture_stderr=True, overwrite_output=True)

        except ffmpeg.Error as e:
            print("\nFFmpeg error output:")
            print(e.stderr.decode())
            raise

        print(f"Successfully created HDR clip at: {output_path}")

    except Exception as e:
        print(f"An error occurred: {str(e)}")

In [ ]:
# start_time = time.time()  # Record start time
# # display_frame2("00:00:20.630")
# display_multiple_frames([
#     "00:00:20.620",
#     "00:00:20.625",
#     "00:00:20.630",
#     "00:00:20.635",
#     "00:00:20.640",
#     "00:00:20.645"
# ])

# end_time = time.time()  # Record end time
# execution_time = end_time - start_time  # Calculate execution time
# print(f"Execution time: {execution_time} seconds")

In [ ]:
# Basic usage with default settings
display_frames_between("00:00:20.63")

In [ ]:
# To cut video between timestamps
if __name__ == "__main__":
    # # Analyze the video first
    # video_info = analyze_video(video_path)

    # if video_info:
    #     # Now use this information to create appropriate cutting parameters
    #     print("\nRecommended FFmpeg parameters based on analysis:")
    #     print("vcodec =", f"'lib{video_info['codec']}'")
    #     print("pix_fmt =", f"'{video_info['pixel_format']}'")
    #     if video_info['bit_rate']:
    #         print("video_bitrate =", f"'{int(int(video_info['bit_rate'])*1.1)}'")  # 10% higher than source)
    #     if video_info['color_space']:
    #         print("colorspace =", f"'{video_info['color_space']}'")
    #     if video_info['color_transfer']:
    #         print("color_trc =", f"'{video_info['color_transfer']}'")
    #     if video_info['color_primaries']:
    #         print("color_primaries =", f"'{video_info['color_primaries']}'")

    # Initialize video configuration once
    initialize_video_config(video_path)

    start_time = "00:00:24.700"  # Start timestamp (hh:mm:ss.fff)
    end_time = "00:00:24.695"    # End timestamp (hh:mm:ss.fff)
    output_filename = "c004.mp4"  # Name of the new clip
    # cut_video(start_time, end_time, output_filename)
    cut_video("00:01:22.700", "00:01:26.895", "c012.mp4")

Cutting video from 00:01:22.700 to 00:01:26.895...
Saving to /content/drive/MyDrive/DMS2024HD/c012.mp4 with HDR preservation...

FFmpeg command:
ffmpeg -ss 82.7 -t 4.194999999999993 -i /content/DMS 2024.MP4 -b:v 22121k -c:a aac -c:v libx265 -color_primaries bt2020 -color_trc arib-std-b67 -colorspace bt2020nc -movflags +faststart -pix_fmt yuv420p10le -preset slow -profile:v main10 /content/drive/MyDrive/DMS2024HD/c012.mp4
Successfully created HDR clip at: /content/drive/MyDrive/DMS2024HD/c012.mp4


In [ ]:
cut_video("00:02:28.700", "00:02:44.965", "c020.mp4")

Cutting video from 00:02:28.700 to 00:02:44.965...
Saving to /content/drive/MyDrive/DMS2024HD/c020.mp4 with HDR preservation...

FFmpeg command:
ffmpeg -ss 148.7 -t 16.265000000000015 -i /content/DMS 2024.MP4 -b:v 22121k -c:a aac -c:v libx265 -color_primaries bt2020 -color_trc arib-std-b67 -colorspace bt2020nc -movflags +faststart -pix_fmt yuv420p10le -preset slow -profile:v main10 /content/drive/MyDrive/DMS2024HD/c020.mp4
Successfully created HDR clip at: /content/drive/MyDrive/DMS2024HD/c020.mp4


In [ ]:
cut_video("00:02:44.970", "00:02:48.930", "c021.mp4")

Cutting video from 00:02:44.970 to 00:02:48.930...
Saving to /content/drive/MyDrive/DMS2024HD/c021.mp4 with HDR preservation...

FFmpeg command:
ffmpeg -ss 164.97 -t 3.960000000000008 -i /content/DMS 2024.MP4 -b:v 22121k -c:a aac -c:v libx265 -color_primaries bt2020 -color_trc arib-std-b67 -colorspace bt2020nc -movflags +faststart -pix_fmt yuv420p10le -preset slow -profile:v main10 /content/drive/MyDrive/DMS2024HD/c021.mp4
Successfully created HDR clip at: /content/drive/MyDrive/DMS2024HD/c021.mp4


In [ ]:
cut_video("00:02:48.935", "00:02:53.130", "c022.mp4")

Cutting video from 00:02:48.935 to 00:02:53.130...
Saving to /content/drive/MyDrive/DMS2024HD/c022.mp4 with HDR preservation...

FFmpeg command:
ffmpeg -ss 168.935 -t 4.194999999999993 -i /content/DMS 2024.MP4 -b:v 22121k -c:a aac -c:v libx265 -color_primaries bt2020 -color_trc arib-std-b67 -colorspace bt2020nc -movflags +faststart -pix_fmt yuv420p10le -preset slow -profile:v main10 /content/drive/MyDrive/DMS2024HD/c022.mp4
Successfully created HDR clip at: /content/drive/MyDrive/DMS2024HD/c022.mp4


In [ ]:
cut_video("00:02:53.135", "00:03:01.495", "c023.mp4")

Cutting video from 00:02:53.135 to 00:03:01.495...
Saving to /content/drive/MyDrive/DMS2024HD/c023.mp4 with HDR preservation...

FFmpeg command:
ffmpeg -ss 173.135 -t 8.360000000000014 -i /content/DMS 2024.MP4 -b:v 22121k -c:a aac -c:v libx265 -color_primaries bt2020 -color_trc arib-std-b67 -colorspace bt2020nc -movflags +faststart -pix_fmt yuv420p10le -preset slow -profile:v main10 /content/drive/MyDrive/DMS2024HD/c023.mp4
Successfully created HDR clip at: /content/drive/MyDrive/DMS2024HD/c023.mp4


In [ ]:
cut_video("00:03:01.500", "00:03:07.195", "c024.mp4")

Cutting video from 00:03:01.500 to 00:03:07.195...
Saving to /content/drive/MyDrive/DMS2024HD/c024.mp4 with HDR preservation...

FFmpeg command:
ffmpeg -ss 181.5 -t 5.694999999999993 -i /content/DMS 2024.MP4 -b:v 22121k -c:a aac -c:v libx265 -color_primaries bt2020 -color_trc arib-std-b67 -colorspace bt2020nc -movflags +faststart -pix_fmt yuv420p10le -preset slow -profile:v main10 /content/drive/MyDrive/DMS2024HD/c024.mp4
Successfully created HDR clip at: /content/drive/MyDrive/DMS2024HD/c024.mp4


In [ ]:
cut_video("00:03:07.200", "00:03:09.350", "c025.mp4")

Cutting video from 00:03:07.200 to 00:03:09.350...
Saving to /content/drive/MyDrive/DMS2024HD/c025.mp4 with HDR preservation...

FFmpeg command:
ffmpeg -ss 187.2 -t 2.1500000000000057 -i /content/DMS 2024.MP4 -b:v 22121k -c:a aac -c:v libx265 -color_primaries bt2020 -color_trc arib-std-b67 -colorspace bt2020nc -movflags +faststart -pix_fmt yuv420p10le -preset slow -profile:v main10 /content/drive/MyDrive/DMS2024HD/c025.mp4
Successfully created HDR clip at: /content/drive/MyDrive/DMS2024HD/c025.mp4


In [ ]:
cut_video("00:02:24.400", "00:02:28.695", "c019.mp4")

Cutting video from 00:02:24.400 to 00:02:28.695...
Saving to /content/drive/MyDrive/DMS2024HD/c019.mp4 with HDR preservation...

FFmpeg command:
ffmpeg -ss 144.4 -t 4.2949999999999875 -i /content/DMS 2024.MP4 -b:v 22121k -c:a aac -c:v libx265 -color_primaries bt2020 -color_trc arib-std-b67 -colorspace bt2020nc -movflags +faststart -pix_fmt yuv420p10le -preset slow -profile:v main10 /content/drive/MyDrive/DMS2024HD/c019.mp4
Successfully created HDR clip at: /content/drive/MyDrive/DMS2024HD/c019.mp4


In [ ]:
cut_video("00:01:19.700", "00:01:22.695", "c011.mp4")

Cutting video from 00:01:19.700 to 00:01:22.695...
Saving to /content/drive/MyDrive/DMS2024HD/c011.mp4 with HDR preservation...

FFmpeg command:
ffmpeg -ss 79.7 -t 2.9949999999999903 -i /content/DMS 2024.MP4 -b:v 22121k -c:a aac -c:v libx265 -color_primaries bt2020 -color_trc arib-std-b67 -colorspace bt2020nc -movflags +faststart -pix_fmt yuv420p10le -preset slow -profile:v main10 /content/drive/MyDrive/DMS2024HD/c011.mp4
Successfully created HDR clip at: /content/drive/MyDrive/DMS2024HD/c011.mp4
